In [114]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import time
import itertools
import re
import psutil
import os
from collections import defaultdict
import pandas as pd
import random

## Model

In [97]:
# ==================================================================================================================== #
# --------------------------------------- CONDITIONAL PROBABILITY TABLES (CPT) --------------------------------------- #
# ==================================================================================================================== #
class CPT:
    def __init__(self, head, parents):
        self.head = head
        self.parents = parents
        self.entries = {}

    def __str__(self):
        comma = ", "
        if len(self.parents) == 0:
            return f"probability ( {self.head.name} ) {{" + "\n" \
                                                            f"  table {comma.join(map(str, self.entries[tuple()].values()))};" + "\n" \
                                                                                                                                 f"}}" + "\n"
        else:
            return f"probability ( {self.head.name} | {comma.join([p.name for p in self.parents])} ) {{" + "\n" + \
                "\n".join([ \
                    f"  ({comma.join(names)}) {comma.join(map(str, values.values()))};" \
                    for names, values in self.entries.items() \
                    ]) + "\n}\n"


# ==================================================================================================================== #
# --------------------------------------------- VARIABLES REPRESENTATION --------------------------------------------- #
# ==================================================================================================================== #
class Variable:
    def __init__(self, name, values):
        self.name = name
        self.values = values
        self.cpt = None

    def __str__(self):
        comma = ", "
        return f"variable {self.name} {{" + "\n" \
            + f"  type discrete [ {len(self.values)} ] {{ {(comma.join(self.values))} }};" + "\n" \
            + f"}}" + "\n"


# ==================================================================================================================== #
# ---------------------------------------------- BAYESIAN NETWORK CLASS ---------------------------------------------- #
# ==================================================================================================================== #
class BayesianNetwork:
    def __init__(self, input_file):
        with open(input_file) as f:
            lines = f.readlines()
        self.variables = {}
        for i in range(len(lines)):
            lines[i] = lines[i].lstrip().rstrip().replace('/', '-')
        i = 0
        while i < len(lines) and not lines[i].startswith("probability"):
            if lines[i].startswith("variable"):
                variable_name = lines[i].rstrip().split(' ')[1]
                i += 1
                variable_def = lines[i].rstrip().split(' ')
                assert (variable_def[1] == 'discrete')
                variable_values = [x for x in variable_def[6:-1]]
                for j in range(len(variable_values)):
                    variable_values[j] = re.sub(r'\(|\)|,', '', variable_values[j])
                variable = Variable(variable_name, variable_values)
                self.variables[variable_name] = variable
            i += 1
        while i < len(lines):
            if lines[i].startswith('probability'):
                split = lines[i].split(' ')
                target_variable_name = split[2]
                variable = self.variables[target_variable_name]
                parents = [self.variables[x.rstrip().lstrip().replace(',', '')] for x in split[4:-2]]
                assert (variable.name == split[2])
                cpt = CPT(variable, parents)
                i += 1
                if len(parents) > 0:
                    nb_lines = 1
                    for p in parents:
                        nb_lines *= len(p.values)
                    for _ in range(nb_lines):
                        cpt_line = [x for x in re.sub(r'\(|\)|,', '', lines[i][:-1]).split()]
                        parent_values = tuple([x for x in cpt_line[:len(parents)]])
                        probabilities = [float(p) for p in cpt_line[len(parents):]]
                        cpt.entries[parent_values] = {v: p for v, p in zip(variable.values, probabilities)}
                        i += 1
                else:
                    cpt_line = [x for x in re.sub(r'\(|\)|,', '', lines[i][:-1]).split()]
                    probabilities = [float(p) for p in cpt_line[1:]]
                    cpt.entries[tuple()] = {v: p for v, p in zip(variable.values, probabilities)}
                variable.cpt = cpt
            i += 1

    def write(self, filename):
        with open(filename, "w") as file:
            for var in self.variables.values():
                file.write(str(var))
            for var in self.variables.values():
                file.write(str(var.cpt))

    def P_Yisy_given_parents_x(self, Y, y, x=tuple()):
        return self.variables[Y].cpt.entries[x][y]

    def P_Yisy_given_parents(self, Y, y, pa={}):
        x = tuple([pa[parent.name] for parent in self.variables[Y].cpt.parents])
        return self.P_Yisy_given_parents_x(Y, y, x)

    def _get_children(self):
        """
        Builds a mapping from each variable to its children in the Bayesian Network.

        Returns:
            dict: A dictionary where each key is a variable name and the value is a list of its children.
        """
        children = defaultdict(list)
        for var in self.variables.values():
            for parent in var.cpt.parents:
                children[parent.name].append(var.name)
        return children

    def _normalize(self, dist):
        """
        Normalizes a probability distribution so that the values sum to 1.

        Args:
            dist (dict): A dictionary mapping values to unnormalized probabilities.

        Returns:
            dict: The normalized distribution, or an empty dict if the total is 0.
        """
        total = sum(dist.values())
        return {k: v / total for k, v in dist.items() if total > 0}

    def _get_pi_contribution(self, pname, pval, pi_msgs, evidence):
        """
        Computes the π-contribution of a parent variable for belief propagation.

        Args:
            pname (str): The name of the parent variable.
            pval (str): The value of the parent variable being considered.
            pi_msgs (dict): Dictionary of π-messages already computed.
            evidence (dict): Observed values for some variables.

        Returns:
            float: The π-contribution for the given parent value.
        """
        if pname in pi_msgs:
            return pi_msgs[pname][pval]
        elif pname in evidence:
            return 1.0 if evidence[pname] == pval else 0.0
        else:
            return 1.0 / len(self.variables[pname].values)

    def _get_vals(self, pname, xi=None, focus_node=None, evidence=None):
        """
        Returns possible values for a parent variable during message passing.

        Args:
            pname (str): The name of the parent variable.
            xi (str, optional): The current value of the focus node (if applicable).
            focus_node (str, optional): The node currently receiving messages.
            evidence (dict): Observed evidence.

        Returns:
            list: List of possible values for pname, based on evidence and context.
        """
        if evidence is None:
            evidence = {}
        if pname == focus_node:
            return [xi]
        elif pname in evidence:
            return [evidence[pname]]
        else:
            return self.variables[pname].values

    def _send_messages_to_root(self, node, evidence, children, lambda_msgs):
        """
        Sends λ-messages from the leaves up to the given node in the Bayesian Network.

        This function recursively computes λ-messages for a node by aggregating messages
        from its children, incorporating evidence when available, as part of belief propagation.

        Args:
            node (str): The variable to which λ-messages are being sent.
            evidence (dict): Observed values for some variables.
            children (dict): A mapping from each variable to its list of children.
            lambda_msgs (dict): Stores already computed λ-messages per variable.

        Returns:
            dict: The λ-message for the given node, mapping each of its values to a likelihood.
        """
        # Avoid redundant computation by reusing cached λ-message
        if node in lambda_msgs:
            return lambda_msgs[node]
        var = self.variables[node]
        lambda_msg = {val: 1.0 for val in var.values}
        for child in children[node]:
            child_lambda = self._send_messages_to_root(child, evidence, children, lambda_msgs)
            child_var = self.variables[child]
            parent_names = [p.name for p in child_var.cpt.parents]
            new_lambda = {}
            for xi in var.values:
                msg = 0.0
                for xj in child_var.values:
                    # Collect possible values for each parent, constrained by evidence and current xi
                    all_pa_vals = itertools.product(
                        *[self._get_vals(pname, xi, node, evidence) for pname in parent_names])
                    for pa in all_pa_vals:
                        pa_dict = dict(zip(parent_names, pa))
                        pa_vals = tuple(pa_dict[p.name] for p in child_var.cpt.parents)
                        prob = child_var.cpt.entries[pa_vals][xj]
                        msg += prob * child_lambda[xj]
                new_lambda[xi] = lambda_msg[xi] * msg
            lambda_msg = new_lambda
        # Override λ-message with hard evidence if the variable is observed
        if node in evidence:
            observed = evidence[node]
            lambda_msg = {val: (1.0 if val == observed else 0.0) for val in var.values}
        lambda_msgs[node] = lambda_msg
        return lambda_msg

    def _send_messages_from_root(self, node, pi_msg, evidence, children, lambda_msgs, beliefs, pi_msgs):
        """
        Sends π-messages from the root to all its descendants in the Bayesian Network.

        This function computes beliefs for all reachable nodes starting from the root,
        using upward (λ) and downward (π) message passing as defined by Pearl's belief propagation algorithm.

        Args:
            node (str): The current variable from which messages are sent.
            pi_msg (dict): The π-message from the parent to the current node (prior belief).
            evidence (dict): Observed values for some variables.
            children (dict): A mapping from each variable to its children.
            lambda_msgs (dict): Precomputed λ-messages from children to parents.
            beliefs (dict): Stores the resulting marginal beliefs per variable.
            pi_msgs (dict): Stores the π-messages sent to each variable.

        Returns:
            None: Updates `beliefs` and `pi_msgs` in place.
        """
        var = self.variables[node]
        lambda_msg = lambda_msgs[node]
        # Combine π and λ messages to compute the final belief for this node
        belief = {val: pi_msg[val] * lambda_msg[val] for val in var.values}
        beliefs[node] = self._normalize(belief)
        pi_msgs[node] = pi_msg
        for child in children[node]:
            child_var = self.variables[child]
            parent_names = [p.name for p in child_var.cpt.parents]
            # Initialize π-message for each possible value of the child
            child_pi = {xj: 0.0 for xj in child_var.values}
            for xj in child_var.values:
                total = 0.0
                # Determine all parent value combinations, constrained by evidence
                all_pa_vals = itertools.product(
                    *[self._get_vals(pname, None, None, evidence) for pname in parent_names])
                for pa in all_pa_vals:
                    pa_dict = dict(zip(parent_names, pa))
                    pa_vals = tuple(pa_dict[p.name] for p in child_var.cpt.parents)
                    prob = child_var.cpt.entries[pa_vals][xj]
                    # Compute contribution of each parent value using upstream π-messages
                    contrib = 1.0
                    for pname, pval in pa_dict.items():
                        contrib *= self._get_pi_contribution(pname, pval, pi_msgs, evidence)
                    total += contrib * prob
                child_pi[xj] = total
            # Recurse on the child node with the computed π-message
            self._send_messages_from_root(child, child_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)

    def query_marginal(self, query_var, evidence):
        """
        Computes the marginal distribution of a variable given observed evidence using belief propagation.

        Args:
            query_var (str): The name of the variable to query.
            evidence (dict): A dictionary mapping observed variable names to their values.

        Returns:
            dict: A probability distribution over the values of `query_var`, normalized to sum to 1.
        """
        children = self._get_children()
        lambda_msgs = {}
        beliefs = {}
        pi_msgs = {}
        root = next(iter(evidence)) if evidence else query_var
        self._send_messages_to_root(root, evidence, children, lambda_msgs)
        root_var = self.variables[root]
        # If root has parents, no prior is available, so assume uniform prior
        if root_var.cpt.parents:
            root_pi = {val: 1.0 for val in root_var.values}
        else:
            root_pi = root_var.cpt.entries[tuple()]
        self._send_messages_from_root(root, root_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)
        # Fallback in case the target query_var wasn't covered in the first propagation
        if query_var not in beliefs:
            self._send_messages_to_root(query_var, evidence, children, lambda_msgs)
            # Again, use uniform prior if the query_var has parents
            if self.variables[query_var].cpt.parents:
                root_pi = {val: 1.0 for val in self.variables[query_var].values}
            else:
                root_pi = self.variables[query_var].cpt.entries[tuple()]
            self._send_messages_from_root(query_var, root_pi, evidence, children, lambda_msgs, beliefs, pi_msgs)
        return beliefs[query_var]

    def query_joint(self, var1, var2, evidence):
        """
        Computes the joint probability distribution of two variables given partial evidence.

        Uses belief propagation to compute:
            P(var1, var2 | evidence) = P(var1 | var2, evidence) * P(var2 | evidence)

        Args:
            var1 (str): The name of the first variable.
            var2 (str): The name of the second variable.
            evidence (dict): Observed values for other variables in the network.

        Returns:
            dict: A nested dictionary {val1: {val2: prob}} representing the normalized joint distribution.
        """
        joint_dist = {}
        for val1 in self.variables[var1].values:
            joint_dist[val1] = {}
            for val2 in self.variables[var2].values:
                extended_evidence = dict(evidence)
                extended_evidence[var1] = val1
                extended_evidence[var2] = val2
                # Fallback in case var2 is not in the marginal or val2 is missing
                marg = self.query_marginal(var2, evidence)
                if var2 not in marg or val2 not in marg:
                    marg = self.query_marginal(var2, evidence)
                if val2 not in marg:
                    continue
                # Fallback in case var1 is not in the conditional or val1 is missing
                cond = self.query_marginal(var1, {**evidence, var2: val2})
                if var1 not in cond or val1 not in cond:
                    cond = self.query_marginal(var1, {**evidence, var2: val2})
                joint_dist[val1][val2] = cond[val1] * marg[val2]
        return self._normalize_nested(joint_dist)

    def _normalize_nested(self, joint_dist):
        """
        Normalizes a nested joint distribution so that all probabilities sum to 1.

        Args:
            joint_dist (dict): Nested dictionary {val1: {val2: prob}}.

        Returns:
            dict: Normalized joint distribution. If total is 0, returns the original.
        """
        # Flatten all probabilities to compute total
        total = sum(v for d in joint_dist.values() for v in d.values())
        if total == 0:
            return joint_dist
        # Normalize each inner value manually by dividing by the total
        for k1 in joint_dist:
            joint_dist[k1] = self._normalize({k2: v / total for k2, v in joint_dist[k1].items()})
        return joint_dist

    def learn_parameters(self, df, laplace_smoothing=True):
        """
        Learns the parameters (CPTs) of the Bayesian network from a DataFrame.

        Args:
            df (pd.DataFrame): The dataset with complete variable assignments (no missing values).
            laplace_smoothing (bool): Whether to apply Laplace smoothing (default: True).

        Returns:
            None: Updates CPTs of each variable in-place.
        """
        alpha = 1 if laplace_smoothing else 0
        for var in self.variables.values():
            # in case of missing CPT
            if var.cpt is None:
                var.cpt = CPT(var, [])
        for var_name, var in self.variables.items():
            if var.cpt is None:
                continue
            parents = var.cpt.parents
            cpt = {}
            parent_names = [p.name for p in parents]
            parent_value_combinations = list(itertools.product(*[self.variables[p].values for p in parent_names]))
            for parent_vals in parent_value_combinations:
                parent_vals = tuple(parent_vals)
                counts = {v: 0 for v in var.values}
                total = 0
                # Select rows where parent values match
                condition = (df[parent_names] == list(parent_vals)).all(axis=1)
                matching_rows = df[condition]
                for y_val in matching_rows[var_name]:
                    if y_val in counts:
                        counts[y_val] += 1
                        total += 1
                smoothed_total = total + alpha * len(var.values)
                probs = {v: (counts[v] + alpha) / smoothed_total for v in var.values}
                cpt[parent_vals] = probs
            if not parents:
                counts = {v: 0 for v in var.values}
                total = 0
                for y_val in df[var_name]:
                    if y_val in counts:
                        counts[y_val] += 1
                        total += 1
                smoothed_total = total + alpha * len(var.values)
                probs = {v: (counts[v] + alpha) / smoothed_total for v in var.values}
                cpt[tuple()] = probs
            var.cpt.entries = cpt


# ==================================================================================================================== #
# ------------------------------------------------- PROJECT PIPELINE ------------------------------------------------- #
# ==================================================================================================================== #
def _clean_missing_dataframe(df):
    """
    Cleans a dataframe containing missing values and numeric strings.

    Replaces "?" with NaN, and converts all non-missing values to stringified integers.

    Args:
        df (pd.DataFrame): The input dataframe with potential missing values and mixed types.

    Returns:
        pd.DataFrame: The cleaned dataframe with consistent string integer values and NaNs.
    """
    df = df.replace("?", pd.NA)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(int(float(x))) if pd.notna(x) else pd.NA)
    return df


def _evaluate_network(bn, test_df, miss_df, testbar):
    """
    Evaluates the accuracy and average confidence of a Bayesian Network on test data with missing values.

    Args:
        bn (BayesianNetwork): The trained Bayesian network.
        test_df (pd.DataFrame): Dataset with complete ground truth values.
        miss_df (pd.DataFrame): Same dataset with some values replaced by NaN.

    Returns:
        float: Accuracy, the proportion of correctly predicted missing values.
        float: Average confidence of the predicted values.
    """
    correct = 0
    total = 0
    confidences = []
    testbar.reset(total=len(miss_df))
    testbar.iterable = range(len(miss_df))
    testbar.set_description("Evaluating")
    start_cpu = time.process_time()
    inferences = 0
    for i in testbar:
        evidence = miss_df.iloc[i].dropna().to_dict()
        ground_truth = test_df.iloc[i].to_dict()
        missing_vars = [col for col in miss_df.columns if pd.isna(miss_df.iloc[i][col])]
        inferences += len(missing_vars)
        for var in missing_vars:
            predicted_dist = bn.query_marginal(var, evidence)
            if not predicted_dist:
                continue
            predicted_val = max(predicted_dist, key=predicted_dist.get)
            confidence = predicted_dist[predicted_val]
            confidences.append(confidence)
            is_correct = (predicted_val == str(ground_truth[var]))
            correct += is_correct
            total += 1
        testbar.update(1)
    accuracy = correct / total if total > 0 else 0.0
    avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0
    return accuracy, avg_confidence, (time.process_time() - start_cpu) / inferences, inferences

## Evaluation

In [110]:
ROOT = "../"
DATA_PATH  = ROOT + "datasets/"
MODEL_PATH = ROOT + "models/"
RESULTS_PATH = ROOT + "results/"
DATASETS = ["alarm", "andes", "asia", "sachs", "sprinkler", "water"]
MODELS = ["bnlearn"]

In [105]:
def compute_model_complexity(bn):
    """
    Computes model complexity metrics for a Bayesian Network:
    number of edges, average degree, depth, and max width.

    Args:
        bn (BayesianNetwork): The network to analyze.

    Returns:
        tuple: (num_edges, avg_degree, depth, width)
    """
    from collections import defaultdict, deque

    # ── Graph representation ─────────────────────────────
    graph = defaultdict(list)
    for child_name, child_var in bn.variables.items():
        if child_var.cpt:
            for parent in child_var.cpt.parents:
                graph[parent.name].append(child_name)

    all_nodes = list(bn.variables.keys())
    num_nodes = len(all_nodes)
    num_edges = sum(len(children) for children in graph.values())
    avg_degree = num_edges / num_nodes if num_nodes > 0 else 0

    # ── Topological traversal to compute depth and width ─
    in_degrees = {v: 0 for v in all_nodes}
    for parent in graph:
        for child in graph[parent]:
            in_degrees[child] += 1

    roots = [v for v, deg in in_degrees.items() if deg == 0]
    depth_dict = {v: 0 for v in roots}
    width_at_depth = defaultdict(int)

    queue = deque(roots)
    while queue:
        node = queue.popleft()
        d = depth_dict[node]
        width_at_depth[d] += 1
        for child in graph[node]:
            depth_dict[child] = d + 1
            queue.append(child)

    max_depth = max(depth_dict.values()) if depth_dict else 0
    max_width = max(width_at_depth.values()) if width_at_depth else 0

    return num_edges, avg_degree, max_depth, max_width

In [106]:
def compute_structural_quality(bn, gt):
    """
    Computes structural quality metrics between a learned Bayesian network and a ground truth:
    Structural Hamming Distance (SHD), edge precision, and edge recall.

    Args:
        bn (BayesianNetwork): Learned Bayesian network.
        gt (BayesianNetwork): Ground truth Bayesian network.

    Returns:
        tuple: (shd, precision, recall)
    """
    def get_edges(net):
        edges = set()
        for child, var in net.variables.items():
            if var.cpt:
                for parent in var.cpt.parents:
                    edges.add((parent.name, child))
        return edges

    edges_bn = get_edges(bn)
    edges_gt = get_edges(gt)

    tp = len(edges_bn & edges_gt)
    fp = len(edges_bn - edges_gt)
    fn = len(edges_gt - edges_bn)
    
    shd = fp + fn
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    return shd, precision, recall

In [111]:
def compute_log_likelihood(bn, df):
    """
    Computes the log-likelihood of a dataset given a Bayesian network.

    Args:
        bn (BayesianNetwork): The Bayesian network with learned parameters.
        df (pd.DataFrame): The complete dataset to evaluate (e.g., test_df).

    Returns:
        float: The average log-likelihood over all rows in the dataset.
    """
    log_likelihood = 0.0
    n = 0

    for _, row in df.iterrows():
        row_dict = row.to_dict()
        row_prob = 1.0

        for var_name, var in bn.variables.items():
            if var.cpt is None:
                continue

            parents = var.cpt.parents
            parent_vals = tuple(row_dict[p.name] for p in parents)
            value = row_dict[var_name]

            # Ensure value is a string if the CPT uses str keys
            prob = var.cpt.entries.get(parent_vals, {}).get(str(value), 1e-9)
            row_prob *= prob

        log_likelihood += np.log(row_prob + 1e-9)  # Add epsilon to avoid log(0)
        n += 1

    return log_likelihood / n if n > 0 else float("-inf")

In [ ]:
def count_model_parameters(bn):
    """
    Counts the total number of free parameters in the Bayesian network.

    Returns:
        int: Total number of conditional probabilities to estimate.
    """
    total = 0
    for var in bn.variables.values():
        if var.cpt is None:
            continue
        num_parent_combinations = 1
        for parent in var.cpt.parents:
            num_parent_combinations *= len(parent.values)
        total += num_parent_combinations * (len(var.values) - 1)  # Last value is determined
    return total


def compute_bic_aic(log_likelihood, num_parameters, num_samples):
    """
    Computes BIC and AIC scores given log-likelihood, number of parameters, and data size.

    Args:
        log_likelihood (float): Average log-likelihood over the dataset.
        num_parameters (int): Number of free parameters in the Bayesian network.
        num_samples (int): Number of rows in the dataset.

    Returns:
        tuple: (bic, aic)
    """
    total_log_likelihood = log_likelihood * num_samples
    bic = -2 * total_log_likelihood + num_parameters * np.log(num_samples)
    aic = -2 * total_log_likelihood + 2 * num_parameters
    return bic, aic

In [ ]:
results = []
process = psutil.Process(os.getpid())

for dataset in tqdm(DATASETS, desc=f"Datasets", position=0):
    # ── Load Data ─────────────────────────────
    train_df = pd.read_csv(DATA_PATH + dataset + "/train.csv")
    test_df = pd.read_csv(DATA_PATH + dataset + "/test.csv")
    miss_df = _clean_missing_dataframe(pd.read_csv(DATA_PATH + dataset + "/test_missing.csv"))

    with tqdm(MODELS, desc=f"Models over `{dataset}`", leave=False, position=1) as evalbar:
        for model in evalbar:
            with tqdm(range(1, 6), desc=f"`{model}` stability", leave=False, position=2) as stabilitybar:
                for i in stabilitybar:
                    # ── Create Network ────────────────────────
                    bn = BayesianNetwork(MODEL_PATH + dataset + "/" + dataset + "_complete.bif")
                    # ── Train ─────────────────────────────────
                    start_cpu = time.process_time()
                    with tqdm(range(1), desc=f"Training", leave=False, position=3) as trainbar:
                        for fold in trainbar:
                            bn.learn_parameters(train_df)
                            trainbar.update(1)
                    training_time = time.process_time() - start_cpu
                    # ── Save Network ────────────────────────
                    bn.write(MODEL_PATH + dataset + "/" + model + "_fold_" + str(i) + ".bif")
                    # ── Model Complexity ──────────────────────────────────
                    with tqdm(range(1), desc=f"Model complexity", leave=False, position=4) as complexitybar:
                        number_edges, degree_avg, depth, width = compute_model_complexity(bn)
                        complexitybar.update(1)
                    # ── Structural quality ──────────────────────────────────
                    with tqdm(range(2), desc=f"Structural quality", leave=False, position=5) as structuralbar:
                        for _ in structuralbar:
                            gt = BayesianNetwork(MODEL_PATH + dataset + "/" + dataset + "_complete.bif")
                            structuralbar.update(1)
                            shd, edge_precision, edge_recall = compute_structural_quality(bn, gt)
                            structuralbar.update(1)
                    # ── Log-likelihood ──────────────────────────────────
                    log_likelihood = compute_log_likelihood(bn, test_df)
                    # ── BIC and AIC ──────────────────────────────────
                    num_parameters = count_model_parameters(bn)
                    bic, aic = compute_bic_aic(log_likelihood, num_parameters, len(test_df))
                    # ── Test ──────────────────────────────────
                    inference_memory = []
                    with tqdm(desc=f"Testing `{model}`", leave=False, position=6) as testbar:
                        before_mem = process.memory_info().rss
                        acc, conf, inf_time, number_inferences = _evaluate_network(bn, test_df, miss_df, testbar)
                        after_mem = process.memory_info().rss
                        memory_used = (after_mem - before_mem) / 1024 / 1024
                        results.append({
                            "dataset": dataset,
                            "model": model,
                            "initialization": i,
                            "train_time_mean": training_time,
                            "edges": number_edges,
                            "avg_degree": degree_avg,
                            "depth_mean": depth,
                            "width_mean": width,
                            "shd": shd,
                            "edge_precision": edge_precision,
                            "edge_recall": edge_recall,
                            "inf_memory_mean": memory_used,
                            "inf_time_mean": inf_time,
                            "inferences": number_inferences,
                            "acc": acc,
                            "conf": conf,
                            "log_likelihood": log_likelihood,
                            "bic": bic,
                            "aic": aic,
                            "num_parameters": num_parameters,
                        })
                        testbar.close()
                    structuralbar.close()
                    complexitybar.close()
                    trainbar.close()
        evalbar.close()

results_df = pd.DataFrame(results
results_df.to_csv(RESULTS_PATH + "models_comparison_over_datasets.csv", index=False)
print(f"Results saved to {RESULTS_PATH}models_comparison_over_datasets.csv")

Datasets:   0%|          | 0/2 [00:00<?, ?it/s]

Models over `alarm`:   0%|          | 0/1 [00:00<?, ?it/s]

`bnlearn` stability:   0%|          | 0/5 [00:00<?, ?it/s]

Training:   0%|          | 0/1 [00:00<?, ?it/s]

Model complexity:   0%|          | 0/1 [00:00<?, ?it/s]

...

Training:   0%|          | 0/1 [00:00<?, ?it/s]

Model complexity:   0%|          | 0/1 [00:00<?, ?it/s]

Structural quality:   0%|          | 0/2 [00:00<?, ?it/s]

Testing `bnlearn`: 0it [00:00, ?it/s]

Results saved to ../results.csv


## Robustness

In [127]:
MODELS = ["complete"]
MISSES = {
    "alarm": 0.5,
    "andes": 0.5,
    "asia": 0.5,
    "sachs": 0.5,
    "sprinkler": 0.5,
    "water": 0.5,
}
CROSS_VALIDATION = 5
STEP = 0.01

for dataset in tqdm(DATASETS, desc="Datasets          ", position=0):
    df = pd.read_csv(DATA_PATH + dataset + "/train.csv")
    results = defaultdict(list)
    total_cells = df.shape[0] * df.shape[1]
    max_missing = int(total_cells * MISSES[dataset])

    for model in tqdm(MODELS, desc=f"Models over `{dataset}`", leave=False, position=1):

        bn = BayesianNetwork(MODEL_PATH + dataset + "/" + model + ".bif")

        for n_missing in tqdm(range(1, max_missing + 1, max(1, int(total_cells * STEP))),
                              desc=f"Model `{model}`  ", leave=False, position=2):
            miss_df = test_df.copy()

            idx = [(i, j) for i in range(miss_df.shape[0]) for j in range(miss_df.shape[1])]
            selected = random.sample(idx, n_missing)
            for i, j in selected:
                miss_df.iat[i, j] = pd.NA

            with tqdm(desc=f"Testing `{model}`", leave=False, position=3) as trainbar:
                acc, conf, _, time = _evaluate_network(bn, test_df, miss_df, trainbar)

            results["dataset"].append(dataset)
            results["model"].append(model)
            results["missing"].append(n_missing)
            results["accuracy"].append(acc)
            results["confidence"].append(conf)
            results["time"].append(time)

results_df = pd.DataFrame(results)
results_df.to_csv(RESULTS_PATH + "robustness.csv", index=False)
print(f"Saved robustness results to: {RESULTS_PATH}robustness.csv")

Datasets          :   0%|          | 0/6 [00:00<?, ?it/s]

Models over `alarm`:   0%|          | 0/1 [00:00<?, ?it/s]

Model `complete`  :   0%|          | 0/50 [00:00<?, ?it/s]

Testing `complete`: 0it [00:00, ?it/s]

KeyError: 'Sprinkler'